In [10]:
import h5py

path = '/home/hulabdl/CMRxRecon2025/mask_radial.h5'

with h5py.File(path) as f:
    print(f.keys())
    print(f['acc16_204x448'][:].shape)

<KeysViewHDF5 ['acc12_116x384', 'acc12_144x512', 'acc12_162x512', 'acc12_168x416', 'acc12_168x448', 'acc12_180x448', 'acc12_204x448', 'acc12_204x512', 'acc12_246x512', 'acc16_116x384', 'acc16_144x512', 'acc16_162x512', 'acc16_168x416', 'acc16_168x448', 'acc16_180x448', 'acc16_204x448', 'acc16_204x512', 'acc16_246x512', 'acc20_116x384', 'acc20_144x512', 'acc20_162x512', 'acc20_168x416', 'acc20_168x448', 'acc20_180x448', 'acc20_204x448', 'acc20_204x512', 'acc20_246x512', 'acc24_116x384', 'acc24_144x512', 'acc24_162x512', 'acc24_168x416', 'acc24_168x448', 'acc24_180x448', 'acc24_204x448', 'acc24_204x512', 'acc24_246x512', 'acc4_116x384', 'acc4_144x512', 'acc4_162x512', 'acc4_168x416', 'acc4_168x448', 'acc4_180x448', 'acc4_204x448', 'acc4_204x512', 'acc4_246x512', 'acc8_116x384', 'acc8_144x512', 'acc8_162x512', 'acc8_168x416', 'acc8_168x448', 'acc8_180x448', 'acc8_204x448', 'acc8_204x512', 'acc8_246x512']>
(12, 204, 448)


In [18]:
path='/home/hulabdl/CMRxRecon2025/ChallengeData/T1rho/TrainingSet/FullSample/Center001/UIH_30T_umr780/P002/T1rho.mat'

import h5py
from scipy.io import loadmat as loadmat_scipy

def loadmat_group(group):
    """
    Load a group in Matlab v7.3 format .mat file using h5py.
    """
    data = {}
    for k, v in group.items():
        if isinstance(v, h5py.Dataset):
            data[k] = v[()]
        elif isinstance(v, h5py.Group):
            data[k] = loadmat_group(v)
    return data

def loadmat(filename):
    """
    Load Matlab v7.3 format .mat file using h5py.
    """
    try:
        with h5py.File(filename, 'r') as f:
            data = {}
            for k, v in f.items():
                if isinstance(v, h5py.Dataset):
                    data[k] = v[()]
                elif isinstance(v, h5py.Group):
                    data[k] = loadmat_group(v)
    except OSError as e:
        f = loadmat_scipy(filename)
        data = {key: value for key, value in f.items() if not key.startswith('__')}
    return data

data = loadmat(path)
print(data.keys())
print(data['kspace'].shape)

# from matplotlib import pyplot as plt
# import numpy as np

# print(np.sum(data['mask'][0], axis=0)[0]-21) 
# plt.imshow(data['mask'][0])
# plt.show()

dict_keys(['kspace'])
(4, 3, 10, 137, 382)


In [ ]:
path='/home/hulabdl/CMRxRecon2025/ChallengeData/LGE/TrainingSet/FullSample/Center001/UIH_30T_umr780/P001/lge_lax_2ch.mat'

import h5py
from scipy.io import loadmat as loadmat_scipy

def loadmat_group(group):
    """
    Load a group in Matlab v7.3 format .mat file using h5py.
    """
    data = {}
    for k, v in group.items():
        if isinstance(v, h5py.Dataset):
            data[k] = v[()]
        elif isinstance(v, h5py.Group):
            data[k] = loadmat_group(v)
    return data

def loadmat(filename):
    """
    Load Matlab v7.3 format .mat file using h5py.
    """
    try:
        with h5py.File(filename, 'r') as f:
            data = {}
            for k, v in f.items():
                if isinstance(v, h5py.Dataset):
                    data[k] = v[()]
                elif isinstance(v, h5py.Group):
                    data[k] = loadmat_group(v)
    except OSError as e:
        f = loadmat_scipy(filename)
        data = {key: value for key, value in f.items() if not key.startswith('__')}
    return data

data = loadmat(path)
print(data['kspace'].shape)

FileNotFoundError: [Errno 2] No such file or directory: ' /home/hulabdl/CMRxRecon2025/ChallengeData/LGE/TrainingSet/FullSample/Center001/UIH_30T_umr780/P001/lge_lax_2ch.mat'